In [1]:
#import xgboost --cannot install
from sklearn.datasets import load_iris
from sklearn.ensemble import (AdaBoostClassifier, 
                              BaggingClassifier, 
                              GradientBoostingRegressor, 
                              RandomForestClassifier, 
                              VotingClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

In [2]:
log_clf = LogisticRegression()
rf_clf = RandomForestClassifier()
svm_clf = SVC()

In [3]:
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rf_clf), ('svc', svm_clf)],
    voting='hard') # hard: majority; soft: highest probability
voting_clf.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [ ]:
for clf in (log_clf, rf_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

### Bagging (Bootstrapped Aggregating) and Pasting (w/o replacement)

In [ ]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=100,
    bootstrap=True,
    n_jobs=-1)

In [ ]:
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

### Out-of-Bag Evaluation

In [4]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=500,
    bootstrap=True,
    n_jobs=-1,
    oob_score=True) # evaluate model with out-of-bag predictions

In [5]:
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

NameError: name 'X_test' is not defined

In [6]:
bag_clf.oob_decision_function_

AttributeError: 'BaggingClassifier' object has no attribute 'oob_decision_function_'

### Random Forests

In [7]:
rnd_clf = RandomForestClassifier(
    n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)
y_pred_rf = rnd_cl.predict(X_test)

NameError: name 'X_train' is not defined

In [8]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(splitter='random', max_leaf_nodes=16),
    n_estimators=500,
    max_samples=1.,
    bootstrap=True,
    n_jobs=-1)

### Feature Importance

In [9]:
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris['data'], iris['target'])
for name, score in zip(iris['feature_names'], 
                       rnd_clf.feature_importances_):
    print(f'{name}: {score:.3f}')

sepal length (cm): 0.099
sepal width (cm): 0.025
petal length (cm): 0.443
petal width (cm): 0.433


In [10]:
tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X, y)

NameError: name 'X' is not defined

In [ ]:
# Train 2nd DecTreeRegressor on 1st one's resids
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

In [ ]:
# etc.
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

In [ ]:
# Ensemble of 3 trees
y_pred = sum(
    tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [11]:
# Same thing:
gbrt = GradientBoostingRegressor(
    max_depth=2, n_estimators=3, learning_rate=1.)
gbrt.fit(X, y)

NameError: name 'X' is not defined

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X, y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)
errors = [mean_squared_errror(y_val, y_pred) 
          for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors) + 1
gbtr_best = GradientBoostingRegressor(max_depth=2, 
                                      n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

NameError: name 'X' is not defined

In [13]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)
min_val_error = float('inf')
error_going_up = 0

for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break # early stopping

NameError: name 'X_train' is not defined

In [17]:
# if you can get xgboost import:
xgb_reg = xgboostXGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

NameError: name 'xgboostXGBRegressor' is not defined

In [18]:
xbg_reg.fit(
    X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=2)

NameError: name 'xbg_reg' is not defined

In [2]:
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris['data'], iris['target'])
for name, score in zip(iris['feature_names'], 
                       rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.09992546060762285
sepal width (cm) 0.022700758093165498
petal length (cm) 0.4546131325309997
petal width (cm) 0.4227606487682119


In [3]:
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                             n_estimators=200,
                             algorithm='SAMME.R',
                             learning_rate=0.5)
ada_clf.fit(X_train, y_train)

NameError: name 'X_train' is not defined

### Gradient Boosting

In [4]:
tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X, y)

NameError: name 'X' is not defined

In [6]:
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

y3 = y2 - tree_reg1.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

y_pred = sum(tree.predict(X_new) 
             for tree in (tree_reg1, tree_reg2, tree_reg3))

NameError: name 'y' is not defined

In [7]:
gbrt = GradientBoostingRegressor(
    max_depth=2, n_estimators=3, learning_rate=1.)
# gbrt.fit(X, y)